### read in random sample (10%) of full df_kickstarter to work with

In [274]:
import pandas as pd
import os
import random
filename = 'raw_data/df_kickstarter.csv'

p = 0.01  # 1% of the lines
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
df = pd.read_csv(filename, header=0,  skiprows=lambda i: i>0 and random.random() > p)
len(df)

6219

#Categorization

In [275]:
import numpy as np
import json

reserve "Cat_Name"(encoding), "backer_count", "blurb"(for NLP),"converted_pledged_amount","country"(encoding),"launched_at"(convert to time),"currency"(encoding),"deadline"(convert to time),"goal","pledged"(delete the ones pledged>goal?),"staff_pick"(meaning?,encoding), "state"(encoding),"usd_pledged","name"(for NLP)

Don't reserve: "Location" is too noisy, too many 1s, "splotlight" is the same as "state","state_changed_at" almost equals to "deadline" 

In [276]:
#extract category
cat_json = df['category'].apply(json.loads)
cat_name=list(map(lambda d: d['slug'], cat_json))
df['Cat_Name']=cat_name

In [277]:
#only study the success and failure cases
df=df.loc[(df['state'] == 'successful') | (df['state'] =='failed')]

In [278]:
#convert the launched_at column and deadline column to months. Add a column to calculate the period of funding
df['launched_at'] = pd.to_datetime(df['launched_at'],unit='s')
df['deadline'] = pd.to_datetime(df['deadline'],unit='s')

In [279]:
df['Launch_deadline_days']=df['deadline']-df['launched_at']
df['Launch_deadline_days'] = df['Launch_deadline_days'].apply(lambda x: x.days)

In [280]:
df['launched_at'] = df['launched_at'].dt.strftime('%m')
df['deadline'] = df['deadline'].dt.strftime('%m')

In [281]:
#df=df.df.dropna(subset=["location"])
#loc_list=list(map(lambda d:json.loads(d)["name"], df['location']))

In [282]:
#df['Loc_Name']=loc_list

In [283]:
features_dict=dict() #inquiry for the actual value represented by the numerical value

In [284]:
#feature engineering for categorical features
for col in df.columns:
    if col in ['Cat_Name', 'country', 'currency', 'staff_pick','state','launched_at','deadline']:
        df[col], indicators = pd.factorize(df[col])
        features_dict[col]=indicators

In [285]:
reserve=['Launch_deadline_days','Cat_Name', 'country', 'currency', 'staff_pick','state',"backers_count","blurb","converted_pledged_amount","goal","launched_at","pledged","usd_pledged","deadline","name"]

In [286]:
df=df[reserve]

In [287]:
df

,Launch_deadline_days,Cat_Name,country,currency,staff_pick,state,backers_count,blurb,converted_pledged_amount,goal,launched_at,pledged,usd_pledged,deadline,name
0,30,0,0,0,0,0,99,We're on a mission to create the ultimate read...,4716,5000.0,0,6493.00,4444.715233,0,Stupid Magazine: A Print Publication about Pop...
1,20,1,1,1,1,0,39,I have big ideas for 2016! But making art is a...,1685,1500.0,1,1685.00,1685.000000,1,"""Untitled"" (2016)"
2,30,2,2,2,1,1,37,Watchee is a tiny smart device that detects an...,1890,15000.0,2,1691.00,1901.059470,2,Watchee - The Smart Watchdog
3,30,3,1,1,1,1,5,"Visual, Audible and Performing Artist Ken Cram...",221,2500.0,3,221.00,221.000000,3,Everybody Drum / Ken Crampton
4,25,4,1,1,1,0,77,The debut of Wolf City Design at the Architect...,8935,7750.0,0,8935.38,8935.380000,0,Wolf City Design- Custom Tile Murals
6,32,5,1,1,1,0,100,We are sharing our love for Arkansas tomatoes ...,6434,5000.0,1,6434.00,6434.000000,1,Arkansas Ketchup Company
7,30,6,1,1,1,0,146,"After 13 years since her last solo project, Un...",5995,5000.0,4,5995.00,5995.000000,4,"KC Groves' new bluegrass & old-time CD ""Happy ..."
8,12,7,1,1,1,1,3,The Story Arts Workshop is dedicated to celebr...,66,350.0,5,66.00,66.000000,5,The Story Arts Workshop 10/29/15 Story Slam
9,30,8,0,0,1,1,2,Our family is opening a food truck to not only...,77,11400.0,6,101.29,76.698939,6,Just Something Different-Family owed food truc...
10,29,9,3,3,1,1,0,Please help me release a track that I'm truly ...,0,800.0,3,0.00,0.000000,7,A SONG FOR WORLDWIDE RELEASE


In [288]:
features_dict["country"][5]

'IT'

In [289]:
df['state'].value_counts() #check the value distribution of a column

0    3481
1    2260
Name: state, dtype: int64

In [290]:
df.dtypes #check the data types of all columns

Launch_deadline_days          int64
Cat_Name                      int64
country                       int64
currency                      int64
staff_pick                    int64
state                         int64
backers_count                 int64
blurb                        object
converted_pledged_amount      int64
goal                        float64
launched_at                   int64
pledged                     float64
usd_pledged                 float64
deadline                      int64
name                         object
dtype: object

In [291]:
#(df['Launch_deadline_days']<0).values.any() #check whether there is funding period < 0

In [292]:
#df.isnull().values.any() #check whether the dataframe has null

In [293]:
#df['current_currency'].unique() #check the the number of different values in a column

Machine Learning Models to Use: Linear Regression, KNN, Decision Tree/Random Forest, PCR/PLS
Measure Metrics: RMSE for regression, Accuracy for classification(pledged > goal, then successful), or more...